In [1]:
import pandas as pd

positive = pd.read_csv("DIJA_REDDIT_top5_pos.csv", engine="python", sep=",", error_bad_lines=False)

In [2]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /Users/Damian/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Damian/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)


positive['cleanText']=positive['Title'].map(lambda s:preprocess(s))

In [5]:
positive.head()

,Unnamed: 0,Date,Open,High,Low,Close,Volume,Adj Close,returns,Volatiluity,Title,cleanText,compound,neg,neu,pos
0,1,39931,8023.560059,8091.660156,7938.979980,8016.950195,274710000,8016.950195,-0.001003,2.183783,"b""Muslims dont care about Swine Flu being call...",muslims dont care swine flu called swine flu i...,-0.8136,0.189,0.622,0.188
1,42,39989,8299.250000,8490.459961,8259.429688,8472.400391,222540000,8472.400391,0.020788,2.363669,"b'Unimaginable Horror In Tehran Today' b""The m...",unimaginable horror tehran today mass protests...,-0.7102,0.217,0.577,0.206
2,107,40084,9663.230469,9823.669922,9663.230469,9789.360352,163780000,9789.360352,0.012847,2.205311,b'Philippine man loses own life after saving 3...,philippine man loses life saving pirate party ...,-0.1157,0.205,0.609,0.186
3,123,40106,10092.419920,10098.389650,9992.809570,10041.480470,214500000,10041.480470,-0.005025,2.023582,"b'""Legalize it"" lobby grows. Mexico and other ...",legalize lobby grows mexico latin american cou...,0.8207,0.158,0.637,0.205
4,606,40806,11401.469730,11550.219730,11373.919920,11408.660160,157060000,11408.660160,0.000671,2.246252,Italy Proposes Law That Will Ban People From T...,italy proposes law ban people internet based s...,0.9750,0.131,0.656,0.213


In [6]:
del positive['compound']
del positive['neg']
del positive['neu']
del positive['pos']

In [13]:
positive.head()

,Unnamed: 0,Date,Open,High,Low,Close,Volume,Adj Close,returns,Volatiluity,Title,cleanText
0,1,39931,8023.560059,8091.660156,7938.979980,8016.950195,274710000,8016.950195,-0.001003,2.183783,"b""Muslims dont care about Swine Flu being call...",muslims dont care swine flu called swine flu i...
1,42,39989,8299.250000,8490.459961,8259.429688,8472.400391,222540000,8472.400391,0.020788,2.363669,"b'Unimaginable Horror In Tehran Today' b""The m...",unimaginable horror tehran today mass protests...
2,107,40084,9663.230469,9823.669922,9663.230469,9789.360352,163780000,9789.360352,0.012847,2.205311,b'Philippine man loses own life after saving 3...,philippine man loses life saving pirate party ...
3,123,40106,10092.419920,10098.389650,9992.809570,10041.480470,214500000,10041.480470,-0.005025,2.023582,"b'""Legalize it"" lobby grows. Mexico and other ...",legalize lobby grows mexico latin american cou...
4,606,40806,11401.469730,11550.219730,11373.919920,11408.660160,157060000,11408.660160,0.000671,2.246252,Italy Proposes Law That Will Ban People From T...,italy proposes law ban people internet based s...


In [14]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [15]:
pip install vaderSentiment 

Note: you may need to restart the kernel to use updated packages.


In [16]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [17]:
preprocess = []
analyzer = SentimentIntensityAnalyzer()
for clean in positive['cleanText']:
    vs = analyzer.polarity_scores(clean)
    preprocess.append(vs)

In [18]:
#positive sentiment: compound score >= 0.05
#neutral sentiment: (compound score > -0.05) and (compound score < 0.05)
#negative sentiment: compound score <= -0.05

In [19]:
df3 = pd.DataFrame(preprocess)

In [20]:
complete_positive = pd.concat([positive,df3], axis=1)

In [21]:
complete_positive

,Unnamed: 0,Date,Open,High,Low,Close,Volume,Adj Close,returns,Volatiluity,Title,cleanText,compound,neg,neu,pos
0,1,39931,8023.560059,8091.660156,7938.979980,8016.950195,274710000,8016.950195,-0.001003,2.183783,"b""Muslims dont care about Swine Flu being call...",muslims dont care swine flu called swine flu i...,-0.8136,0.189,0.622,0.188
1,42,39989,8299.250000,8490.459961,8259.429688,8472.400391,222540000,8472.400391,0.020788,2.363669,"b'Unimaginable Horror In Tehran Today' b""The m...",unimaginable horror tehran today mass protests...,-0.7102,0.217,0.577,0.206
2,107,40084,9663.230469,9823.669922,9663.230469,9789.360352,163780000,9789.360352,0.012847,2.205311,b'Philippine man loses own life after saving 3...,philippine man loses life saving pirate party ...,-0.1157,0.205,0.609,0.186
3,123,40106,10092.419920,10098.389650,9992.809570,10041.480470,214500000,10041.480470,-0.005025,2.023582,"b'""Legalize it"" lobby grows. Mexico and other ...",legalize lobby grows mexico latin american cou...,0.8207,0.158,0.637,0.205
4,606,40806,11401.469730,11550.219730,11373.919920,11408.660160,157060000,11408.660160,0.000671,2.246252,Italy Proposes Law That Will Ban People From T...,italy proposes law ban people internet based s...,0.9750,0.131,0.656,0.213
5,1419,41985,17590.050780,17590.050780,17280.830080,17280.830080,121950000,17280.830080,-0.017930,2.490269,Canada just made it legal for police to search...,canada made legal police search cellphone with...,-0.3797,0.166,0.647,0.188


In [22]:
complete_positive.to_csv('top_5_positive.csv')